# Control

In this notebook we want to control the chaos in the Henon map. The Henon map is defined by

$$
\begin{align}
x_{n+1}&=1-ax_n^2+y_n\\
y_{n+1}&=bx_n
\end{align}.
$$

In [40]:
from plotly import offline as py
from plotly import graph_objs as go

py.init_notebook_mode(connected=True)

### Fixed points

First we need to find the fixed points of the Henon map. From $y_n=y_{n+1}=bx_n$ we can elliminate $y_n$ in the first equation. The quadratic equation obtained after ellimination with $x_n=x_{n+1}$ yields,
$$
\begin{align}
x^*=\frac{b-1\pm\sqrt{4a+(b-1)^2}}{2a},
&&
y^*=bx^*,
\end{align}
$$
as the fixed points of the Henon map.

In [41]:
def henon_map(x0, y0, a, b, N):
    x = [x0]
    y = [y0]
    
    for i in range(N):
        xn = x[-1]
        yn = y[-1]
        
        x.append(1 - a * xn**2 + yn)
        y.append(b * xn)
        
    return x, y

def fixed_points(a, b):
    u = (b - 1) / (2 * a)
    v = np.sqrt(4 * a + (b - 1)**2) / (2 * a)
    
    x1 = u - v
    x2 = u + v
    
    y1 = b * x1
    y2 = b * x2
    
    return [(x1, y1), (x2, y2)]

In [42]:
((xf1, yf1), (xf2, yf2)) = fixed_points(a=1.4, b=0.3)

radius = 0.1

layout = go.Layout(
    title='Henon Attractor',
    xaxis=dict(title='x'),
    yaxis=dict(title='y', scaleanchor='x'),
    showlegend=False,
    shapes=[
        {
            'type': 'circle',
            'xref': 'x',
            'yref': 'y',
            'x0': xf1 + radius,
            'y0': yf1 + radius,
            'x1': xf1 - radius,
            'y1': yf1 - radius,
            'line': { 'color': 'gray' },
        },
        {
            'type': 'circle',
            'xref': 'x',
            'yref': 'y',
            'x0': xf2 + radius,
            'y0': yf2 + radius,
            'x1': xf2 - radius,
            'y1': yf2 - radius,
        },
    ]
)

x = []
y = []

for i in range(50):
    x0, y0 = np.random.uniform(0.2, 0.8, 2)
    
    xx, yy = henon_map(x0, y0, a=1.4, b=0.3, N=100)
    
    if np.abs(xx[-1]) < 10 and np.abs(yy[-1]) < 10:
        x += xx
        y += yy

figure = go.Figure([
    go.Scatter(x=x, y=y, mode='markers', marker=dict(size=3))
], layout)

py.iplot(figure)

So the second fixed point (positive sign) sits on the attractor.

In [43]:
def fixed_point(a, b):
    return fixed_points(a, b)[1]

fixed_point(a=1.4, b=0.3)

(0.6313544770895047, 0.1894063431268514)

We assume that coordinates and parameters are sufficiently close such that the following Taylor expansion is valid,$$
\boldsymbol{x}_{n+1}
=
\boldsymbol{F}\left(\boldsymbol{x}^*,\boldsymbol{r}_0\right)
+
\frac{d\boldsymbol{F}}{d\boldsymbol{x}_n}\Bigr|_{\boldsymbol{x}^*,\boldsymbol{r}_0}\left(\boldsymbol{x}_n-\boldsymbol{x}^*\right)
+
\frac{d\boldsymbol{F}}{d\boldsymbol{r}_n}\Bigr|_{\boldsymbol{x}^*,\boldsymbol{r}_0}\left(\boldsymbol{r}_n-\boldsymbol{r}_0\right).$$

In the regime where these linear approximations are valid we can use, $$
\Delta\boldsymbol{r}_n
=
\gamma\left(\boldsymbol{x}_n-\boldsymbol{x}^*\right). $$
Further introducing $\Delta\boldsymbol{x}_n=\boldsymbol{x}_n-\boldsymbol{x}^*$ we can rewrite the map as, $$
\Delta\boldsymbol{x}_{n+1}
=
\underbrace{\left(
\frac{d\boldsymbol{F}}{d\boldsymbol{x}_n}\Bigr|_{\boldsymbol{x}^*,\boldsymbol{r}_0}
+
\frac{d\boldsymbol{F}}{d\boldsymbol{r}_n}\Bigr|_{\boldsymbol{x}^*,\boldsymbol{r}_0}
\right)}_{A}
\Delta\boldsymbol{x}_n.$$

The Jacobians are $$
\begin{align}
\frac{d\boldsymbol{F}}{d\boldsymbol{x}_n}\Bigr|_{\boldsymbol{x}^*,\boldsymbol{r}_0}
=
\begin{pmatrix}
-2 a_0 x^* & 1 \\
b_0 & 0
\end{pmatrix},
&&
\frac{d\boldsymbol{F}}{d\boldsymbol{r}_n}\Bigr|_{\boldsymbol{x}^*,\boldsymbol{r}_0}
=
\begin{pmatrix}
-{x^*}^2 & 0 \\
0 & x^*
\end{pmatrix}
\end{align}. $$

Thus the matrix $A$ reads, $$
A
=
\begin{pmatrix}
-2a_0x^*-\gamma{x^*}^2 & 1 \\
b_0 & \gamma x^*
\end{pmatrix}.
$$ The optimal value for $\gamma$ can be found for $0=A\Delta\boldsymbol{x}_n$.

In [35]:
def eigenvector(a, b):
    xf, yf = fixed_point(a, b)
    
    A = np.array([
        [-2 * a * xf - xf**2, 1],
        [b, xf]
    ])
    
    return u-v, u+v

eigenvalues(a=1.4, b=0.3)

(-1.9237388581534067, 0.15594632230279393)


The Jacobian of the Henon map close to $(x^*,a_0,b_0)$ is given through, $$
\begin{pmatrix}
-2 a_0 x^* & 1 \\
b_0 & 0
\end{pmatrix},$$
and has eigenvalues $$\lambda=-a_0\left[x^*\pm\sqrt{{x^*}^2+b_0/a_0^2}\right]$$

In [36]:
fixed_point(a=1.4, b=0.3)

(0.6313544770895047, 0.1894063431268514)